In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
data = data.dropna()

data.head()

In [ ]:
#use open to predict close
df_open = data['Open']
open_plt = plt.axes()
open_plt.plot(df_open, label='open', color='green')

In [ ]:
df_close = data['Close']
close_plt = plt.axes()
close_plt.plot(df_close, label='close', color='red')

In [ ]:
# spliting test data
split_dig = round(len(data)*0.2)
train = df_open[:25000]
test_1 = df_open[25000:35000]
test_2 = df_open[:10000]

print(len(train),len(test_1),len(test_2))

train = np.array(train)
test_1 = np.array(test_1)
test_2 = np.array(test_2)

def process_data(data, lag):
    x = []
    y = []
    for i in range(lag, len(data)):
        x.append(data[i-lag:i])
        y.append(data[i])
    return np.array(x), np.array(y)

# 7 days data to predict 1 day
predict_days = 60
train_x, train_y = process_data(train, predict_days)
test1_x, test1_y = process_data(test_1, predict_days)
test2_x, test2_y = process_data(test_2, predict_days)

scale = MinMaxScaler(feature_range=(0,1))

train_x = scale.fit_transform(train_x)
test1_x = scale.transform(test1_x)
test2_x = scale.transform(test2_x)

train_x = np.reshape(train_x, (train_x.shape[0],train_x.shape[1],1))
test1_x = np.reshape(test1_x, (test1_x.shape[0],test1_x.shape[1],1))
test2_x = np.reshape(test2_x, (test2_x.shape[0],test2_x.shape[1],1))


print(train_x.shape)
print(test1_x.shape)
print(test2_x.shape)

In [ ]:
# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(train_x.shape[1],1)))
regressor.add(Dropout(0.2))
# Second LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Third LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Fourth LSTM layer
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
# The output layer
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='rmsprop',loss='mean_squared_error')
# Fitting to the training set
history = regressor.fit(train_x,train_y,epochs=5,batch_size=32)

In [ ]:
plt.title('train_loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.plot( history.history["loss"])

In [ ]:
regressor.save('lstm_model_1.h5')

In [ ]:
pred = regressor.predict(test1_x)

pred_plt = plt.axes()
pred_plt.plot(pred, label='pred', color='red')
pred_plt.plot(test1_y, label='real', color='blue')

In [ ]:
pred = regressor.predict(test2_x)

pred_plt = plt.axes()
pred_plt.plot(pred, label='pred', color='red')
pred_plt.plot(test2_y, label='real', color='blue')